In [1]:
import warnings
warnings.filterwarnings("ignore")

# Basics - Data Handling
import numpy as np
import pandas as pd

# sklearn Classification and Model Building
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import StackingClassifier

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Cross Validation
from sklearn.model_selection import GridSearchCV

# Scoring functions
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Read Data

In [2]:
X_train = pd.read_csv("train_smote.csv", index_col=0)
y_train = X_train["y"]
X_train.drop(["y"], axis=1, inplace=True)
X_train.head(2)

,Income,Age,Score1,Score3,Score4,Score5,Loan type_1.0,Occupation type_1.0,Occupation type_2.0
0,16167.84409,1.0,0.272247,7.793966,602.632122,3396.474235,0,1,0
1,15772.85155,0.0,0.203328,12.165225,600.397183,3476.857709,0,1,0


## Grid Search with 5-fold CV

### [MLP](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [3]:
# GridSearch for MLP
parameters = {'hidden_layer_sizes':[(50,),(100,),(200,)], 'learning_rate':['constant', 'adaptive'], 'early_stopping':[True, False]}
model = MLPClassifier(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/mlp_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_early_stopping,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,25.098474,16.053974,0.063223,0.022155,True,"(50,)",constant,"{'early_stopping': True, 'hidden_layer_sizes':...",0.853139,0.849828,0.798636,0.767051,0.844149,0.822560,0.034054,5
1,23.705503,19.362145,0.054540,0.010724,True,"(50,)",adaptive,"{'early_stopping': True, 'hidden_layer_sizes':...",0.853139,0.849828,0.798636,0.767051,0.844149,0.822560,0.034054,5
2,38.134837,16.407413,0.076496,0.009457,True,"(100,)",constant,"{'early_stopping': True, 'hidden_layer_sizes':...",0.822172,0.818820,0.870929,0.870312,0.853360,0.847119,0.022658,3
3,36.534593,16.084879,0.071008,0.005726,True,"(100,)",adaptive,"{'early_stopping': True, 'hidden_layer_sizes':...",0.822172,0.818820,0.870929,0.870312,0.853360,0.847119,0.022658,3
4,42.887368,16.839749,0.126644,0.031307,True,"(200,)",constant,"{'early_stopping': True, 'hidden_layer_sizes':...",0.832788,0.864191,0.846472,0.858638,0.849779,0.850374,0.010808,1
5,35.178979,11.906039,0.114662,0.010639,True,"(200,)",adaptive,"{'early_stopping': True, 'hidden_layer_sizes':...",0.832788,0.864191,0.846472,0.858638,0.849779,0.850374,0.010808,1
6,21.746652,9.458713,0.052868,0.013126,False,"(50,)",constant,"{'early_stopping': False, 'hidden_layer_sizes'...",0.796195,0.794296,0.731811,0.864848,0.702326,0.777895,0.056567,9
7,21.720733,9.432673,0.050606,0.012771,False,"(50,)",adaptive,"{'early_stopping': False, 'hidden_layer_sizes'...",0.796195,0.794296,0.731811,0.864848,0.702326,0.777895,0.056567,9
8,58.581303,25.168430,0.087666,0.005418,False,"(100,)",constant,"{'early_stopping': False, 'hidden_layer_sizes'...",0.713979,0.842584,0.868790,0.716072,0.862208,0.800727,0.070507,7
9,58.504849,24.835646,0.091707,0.005051,False,"(100,)",adaptive,"{'early_stopping': False, 'hidden_layer_sizes'...",0.713979,0.842584,0.868790,0.716072,0.862208,0.800727,0.070507,7


## MLP with Pipeline

In [3]:
# GridSearch for MLP, Standard Scaling
parameters = {'mlp__hidden_layer_sizes':[(50,),(100,),(200,)], 'mlp__learning_rate':['constant', 'adaptive'], 'mlp__early_stopping':[True, False]}
model = Pipeline([('scaler', StandardScaler()), ('mlp', MLPClassifier(max_iter=1000, random_state=42))])

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/svm_stdscaler_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__early_stopping,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,56.716128,12.331112,0.057226,0.002255,True,"(50,)",constant,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.972710,0.983026,0.985501,0.985061,0.980489,0.981358,0.004673,11
1,53.746583,11.110353,0.059937,0.003669,True,"(50,)",adaptive,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.972710,0.983026,0.985501,0.985061,0.980489,0.981358,0.004673,11
2,64.342840,8.412649,0.130179,0.042848,True,"(100,)",constant,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.975775,0.985116,0.986898,0.986977,0.985396,0.984032,0.004197,9
3,64.849811,8.490849,0.079901,0.005098,True,"(100,)",adaptive,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.975775,0.985116,0.986898,0.986977,0.985396,0.984032,0.004197,9
4,131.959674,38.129003,0.165556,0.013582,True,"(200,)",constant,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.974061,0.987893,0.987389,0.986763,0.987176,0.984656,0.005310,7
5,138.898780,36.045442,0.153762,0.007198,True,"(200,)",adaptive,"{'mlp__early_stopping': True, 'mlp__hidden_lay...",0.974061,0.987893,0.987389,0.986763,0.987176,0.984656,0.005310,7
6,222.512570,43.212444,0.056137,0.012070,False,"(50,)",constant,"{'mlp__early_stopping': False, 'mlp__hidden_la...",0.975899,0.988058,0.986808,0.988863,0.987790,0.985484,0.004837,5
7,165.479733,31.134830,0.047117,0.000980,False,"(50,)",adaptive,"{'mlp__early_stopping': False, 'mlp__hidden_la...",0.975899,0.988058,0.986808,0.988863,0.987790,0.985484,0.004837,5
8,171.330596,25.329152,0.089663,0.002711,False,"(100,)",constant,"{'mlp__early_stopping': False, 'mlp__hidden_la...",0.979153,0.987858,0.990039,0.990924,0.988958,0.987386,0.004244,1
9,188.585484,40.738093,0.087512,0.005033,False,"(100,)",adaptive,"{'mlp__early_stopping': False, 'mlp__hidden_la...",0.979153,0.987858,0.990039,0.990924,0.988958,0.987386,0.004244,1


### [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [4]:
# GridSearch for Random Forest
parameters = {'max_depth':[None, 10, 7, 5], 'min_samples_split':[2, 3, 4]}
model = RandomForestClassifier(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/rf_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,14.661149,0.543857,0.282420,0.001868,None,2,"{'max_depth': None, 'min_samples_split': 2}",0.975983,0.988676,0.988892,0.989162,0.988609,0.986264,0.005145,1
1,14.151201,0.155637,0.282341,0.002421,None,3,"{'max_depth': None, 'min_samples_split': 3}",0.975347,0.988809,0.988802,0.988760,0.988251,0.985994,0.005327,3
2,14.029558,0.270836,0.281258,0.001233,None,4,"{'max_depth': None, 'min_samples_split': 4}",0.976111,0.988894,0.988667,0.988625,0.987850,0.986029,0.004972,2
3,11.608537,0.138247,0.210832,0.001959,10,2,"{'max_depth': 10, 'min_samples_split': 2}",0.959295,0.974543,0.975272,0.974315,0.973908,0.971467,0.006102,6
4,11.591411,0.053109,0.211505,0.002305,10,3,"{'max_depth': 10, 'min_samples_split': 3}",0.959496,0.974679,0.975415,0.974550,0.974643,0.971757,0.006138,5
5,11.619734,0.044456,0.213031,0.001405,10,4,"{'max_depth': 10, 'min_samples_split': 4}",0.959832,0.974861,0.975094,0.974763,0.975098,0.971930,0.006050,4
6,9.182825,0.074187,0.163752,0.001742,7,2,"{'max_depth': 7, 'min_samples_split': 2}",0.940035,0.955387,0.956463,0.954504,0.954238,0.952125,0.006095,9
7,9.177744,0.099253,0.164474,0.001398,7,3,"{'max_depth': 7, 'min_samples_split': 3}",0.940701,0.955787,0.956486,0.954350,0.954385,0.952342,0.005878,8
8,9.207544,0.059649,0.165308,0.003695,7,4,"{'max_depth': 7, 'min_samples_split': 4}",0.941020,0.956231,0.956020,0.954360,0.954372,0.952401,0.005745,7
9,7.250213,0.039612,0.132504,0.001151,5,2,"{'max_depth': 5, 'min_samples_split': 2}",0.912865,0.934616,0.939679,0.933382,0.935230,0.931154,0.009389,10


### [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

In [5]:
# GridSearch for AdaBoost
parameters = {'algorithm':['SAMME', 'SAMME.R']}
model = AdaBoostClassifier(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/adaboost_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.864534,0.004231,0.040461,0.000863,SAMME,{'algorithm': 'SAMME'},0.882570,0.913960,0.918008,0.917557,0.922554,0.910930,0.01444,2
1,4.137202,0.020555,0.091410,0.000933,SAMME.R,{'algorithm': 'SAMME.R'},0.905616,0.931657,0.932688,0.931048,0.932133,0.926629,0.01052,1


### [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [6]:
# GridSearch for SVM
parameters = {'C':[1, 10, 50, 100, 200, 500], 'gamma':['scale', 'auto']}
model = SVC(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/svm_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,249.895306,34.397118,28.366369,1.057857,1,scale,"{'C': 1, 'gamma': 'scale'}",0.709638,0.702923,0.707484,0.709113,0.708854,0.707602,0.002445,12
1,874.688312,65.030553,38.778030,1.411103,1,auto,"{'C': 1, 'gamma': 'auto'}",0.941614,0.922296,0.924996,0.919779,0.923339,0.926405,0.007791,6
2,201.943494,18.147250,23.965464,2.152131,10,scale,"{'C': 10, 'gamma': 'scale'}",0.746713,0.742399,0.745507,0.742988,0.744192,0.744360,0.001588,11
3,1279.955099,245.119140,42.646536,10.919885,10,auto,"{'C': 10, 'gamma': 'auto'}",0.947184,0.930161,0.931272,0.927914,0.930486,0.933403,0.006980,5
4,254.662306,26.848938,33.635796,11.250959,50,scale,"{'C': 50, 'gamma': 'scale'}",0.764183,0.761439,0.768448,0.765451,0.766335,0.765171,0.002327,10
5,1215.209648,58.327885,38.045802,1.979106,50,auto,"{'C': 50, 'gamma': 'auto'}",0.947184,0.930204,0.931316,0.927914,0.930492,0.933422,0.006973,1
6,187.530979,8.706294,21.712277,0.952282,100,scale,"{'C': 100, 'gamma': 'scale'}",0.779614,0.774060,0.782112,0.779799,0.778215,0.778760,0.002662,9
7,1580.359100,616.074913,38.371447,1.841673,100,auto,"{'C': 100, 'gamma': 'auto'}",0.947184,0.930204,0.931316,0.927914,0.930492,0.933422,0.006973,1
8,167.867401,14.039925,20.295518,0.799807,200,scale,"{'C': 200, 'gamma': 'scale'}",0.794390,0.788654,0.796328,0.794311,0.791212,0.792979,0.002713,8
9,1164.975708,111.272714,36.798545,0.188233,200,auto,"{'C': 200, 'gamma': 'auto'}",0.947184,0.930204,0.931316,0.927914,0.930492,0.933422,0.006973,1


## [SVM with Pipeline](https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html)

In [15]:
# GridSearch for SVM, StandardScaler
parameters = {'svc__C':[1, 10, 50, 100, 200, 500], 'svc__gamma':['scale', 'auto']}
model = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/svm_stdscaler_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,47.091523,1.959452,6.718209,0.241738,1,scale,"{'svc__C': 1, 'svc__gamma': 'scale'}",0.959391,0.974181,0.974588,0.972775,0.973455,0.970878,0.005777,11
1,47.255894,2.064197,6.728055,0.248090,1,auto,"{'svc__C': 1, 'svc__gamma': 'auto'}",0.959391,0.974181,0.974588,0.972775,0.973455,0.970878,0.005777,11
2,42.486882,2.751050,4.954990,0.221136,10,scale,"{'svc__C': 10, 'svc__gamma': 'scale'}",0.964014,0.978964,0.979421,0.978614,0.979577,0.976118,0.006061,9
3,42.232484,2.550916,4.939200,0.213460,10,auto,"{'svc__C': 10, 'svc__gamma': 'auto'}",0.964014,0.978964,0.979421,0.978614,0.979577,0.976118,0.006061,9
4,54.695873,4.160201,4.199658,0.204386,50,scale,"{'svc__C': 50, 'svc__gamma': 'scale'}",0.966642,0.981913,0.982194,0.982295,0.981861,0.978981,0.006172,7
5,55.331222,3.707271,4.193846,0.210306,50,auto,"{'svc__C': 50, 'svc__gamma': 'auto'}",0.966642,0.981913,0.982194,0.982295,0.981861,0.978981,0.006172,7
6,71.051963,5.908175,3.953083,0.217666,100,scale,"{'svc__C': 100, 'svc__gamma': 'scale'}",0.966890,0.982412,0.982827,0.983156,0.982409,0.979539,0.006330,5
7,70.613013,6.142638,4.004085,0.252209,100,auto,"{'svc__C': 100, 'svc__gamma': 'auto'}",0.966890,0.982412,0.982827,0.983156,0.982409,0.979539,0.006330,5
8,96.513708,10.038228,3.760772,0.204916,200,scale,"{'svc__C': 200, 'svc__gamma': 'scale'}",0.967983,0.982911,0.983680,0.983387,0.982955,0.980183,0.006107,3
9,98.800117,10.782420,3.764188,0.216594,200,auto,"{'svc__C': 200, 'svc__gamma': 'auto'}",0.967983,0.982911,0.983680,0.983387,0.982955,0.980183,0.006107,3


## [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [10]:
# GridSearch for KNN
parameters = {'n_neighbors':[3, 5, 10, 15]}
model = KNeighborsClassifier()

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/knn_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.164193,0.012257,0.621581,0.008646,3,{'n_neighbors': 3},0.947545,0.945629,0.945792,0.943784,0.946859,0.945922,0.001280,1
1,0.159811,0.005542,0.637587,0.004404,5,{'n_neighbors': 5},0.939427,0.936598,0.938453,0.936732,0.938970,0.938036,0.001162,2
2,0.169510,0.029043,0.697113,0.009239,10,{'n_neighbors': 10},0.927730,0.926166,0.927975,0.925185,0.927740,0.926959,0.001096,3
3,0.155948,0.006326,0.735752,0.008581,15,{'n_neighbors': 15},0.912225,0.910997,0.913341,0.910596,0.912829,0.911998,0.001050,4


## [KNN with Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [9]:
# GridSearch for KNN with StandardScaler
parameters = {'knn__n_neighbors':[3, 5, 10, 15]}
model = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/knn_stdscaler_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.183784,0.043306,1.267711,0.173951,3,{'knn__n_neighbors': 3},0.977014,0.983173,0.985323,0.985242,0.985117,0.983174,0.003182,1
1,0.146889,0.000677,1.260590,0.024357,5,{'knn__n_neighbors': 5},0.974768,0.982987,0.983191,0.983519,0.982980,0.981489,0.003366,2
2,0.200696,0.063840,2.472370,0.491078,10,{'knn__n_neighbors': 10},0.967971,0.980022,0.979872,0.980198,0.976977,0.977008,0.004672,3
3,0.194164,0.044959,2.331671,0.514615,15,{'knn__n_neighbors': 15},0.966383,0.978157,0.977808,0.979008,0.975581,0.975387,0.004642,4


## [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [7]:
# GridSearch for DecisionTreeClassifier
parameters = {'criterion':['gini', 'entropy'], 'splitter':['best', 'random'], 'max_depth':[None, 10, 7, 5], 'min_samples_split':[2, 3, 4]}
model = DecisionTreeClassifier(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/decisiontree_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.870192,0.039075,0.015427,0.000447,gini,None,2,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.965157,0.975895,0.979059,0.978786,0.976370,0.975053,0.005106,4
1,0.098385,0.003857,0.016156,0.000502,gini,None,2,random,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.963093,0.976477,0.977161,0.978156,0.975789,0.974135,0.005576,9
2,0.864684,0.033987,0.016501,0.002428,gini,None,3,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.963649,0.975707,0.979181,0.979228,0.977090,0.974971,0.005815,6
3,0.094014,0.004170,0.016257,0.000498,gini,None,3,random,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.960673,0.976117,0.975082,0.976862,0.976452,0.973037,0.006210,12
4,0.864012,0.034132,0.015935,0.001243,gini,None,4,best,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.963686,0.975484,0.979182,0.978387,0.976920,0.974732,0.005666,7
5,0.093154,0.001071,0.016826,0.001901,gini,None,4,random,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.960471,0.977602,0.977560,0.977194,0.975894,0.973744,0.006665,10
6,0.705289,0.015386,0.014792,0.000539,gini,10,2,best,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.950992,0.967870,0.969446,0.967008,0.968340,0.964731,0.006914,15
7,0.069547,0.002379,0.015008,0.000504,gini,10,2,random,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.908957,0.924562,0.913287,0.932733,0.939999,0.923908,0.011607,28
8,0.703112,0.017650,0.014674,0.000513,gini,10,3,best,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.951184,0.967951,0.969443,0.966956,0.968645,0.964836,0.006875,14
9,0.071421,0.004285,0.015800,0.001176,gini,10,3,random,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.904183,0.905610,0.934201,0.926759,0.933489,0.920848,0.013289,30


## [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)

In [8]:
# GridSearch for SGDClassifier
parameters = {'penalty':['l2','l1','elasticnet'], 'learning_rate':['constant','optimal','invscaling','adaptive'], 'eta0':[0.001, 0.05, 0.01, 0.1, 0.5, 1]}
model = SGDClassifier(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("parameter_search/sgd_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta0,param_learning_rate,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.229324,0.294640,0.019408,0.000845,0.001,constant,l2,"{'eta0': 0.001, 'learning_rate': 'constant', '...",0.330607,0.666885,0.774907,0.594648,0.000535,0.473516,0.278221,65
1,1.640125,0.277214,0.019860,0.000491,0.001,constant,l1,"{'eta0': 0.001, 'learning_rate': 'constant', '...",0.765391,0.782056,0.674010,0.526796,0.734298,0.696510,0.092520,56
2,1.510267,0.242644,0.019064,0.000769,0.001,constant,elasticnet,"{'eta0': 0.001, 'learning_rate': 'constant', '...",0.731000,0.667005,0.671432,0.591399,0.670910,0.666349,0.044372,61
3,7.730865,0.431817,0.021299,0.002095,0.001,optimal,l2,"{'eta0': 0.001, 'learning_rate': 'optimal', 'p...",0.800019,0.789422,0.771414,0.651949,0.801834,0.762928,0.056533,35
4,4.827232,1.046514,0.020083,0.000394,0.001,optimal,l1,"{'eta0': 0.001, 'learning_rate': 'optimal', 'p...",0.786147,0.786332,0.796811,0.795890,0.796502,0.792336,0.004987,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,7.994716,0.510252,0.020455,0.000352,1,invscaling,l1,"{'eta0': 1, 'learning_rate': 'invscaling', 'pe...",0.705641,0.797634,0.694284,0.793780,0.803204,0.758909,0.048356,41
68,7.882533,1.350218,0.020370,0.000319,1,invscaling,elasticnet,"{'eta0': 1, 'learning_rate': 'invscaling', 'pe...",0.364642,0.780651,0.789544,0.791894,0.694294,0.684205,0.163840,57
69,8.281173,1.826273,0.020737,0.000497,1,adaptive,l2,"{'eta0': 1, 'learning_rate': 'adaptive', 'pena...",0.801586,0.798152,0.803637,0.803441,0.800469,0.801457,0.002030,4
70,5.553568,0.476826,0.020386,0.000893,1,adaptive,l1,"{'eta0': 1, 'learning_rate': 'adaptive', 'pena...",0.795746,0.790026,0.798205,0.794181,0.796586,0.794949,0.002784,18


## [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [13]:
# GridSearch for LogisticRegression
parameters = {'penalty':['l2','l1','elasticnet'], 'C':[1, 10, 50, 100, 200, 500], 'solver':['newton-cg','lbfgs','liblinear','sag','saga']}
model = LogisticRegression(random_state=42)

clf = GridSearchCV(model, parameters, scoring="f1")
clf.fit(X_train, y_train)
df = pd.DataFrame(clf.cv_results_)
df.to_csv("logistic_param_tuning.csv")
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,12.913981,0.690523,0.022508,0.005156,1,l2,newton-cg,"{'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}",0.854698,0.875867,0.879518,0.880180,0.876492,0.873351,0.009474,5
1,1.283200,0.398664,0.020772,0.000548,1,l2,lbfgs,"{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}",0.804313,0.811924,0.816862,0.804215,0.815042,0.810471,0.005309,24
2,1.028832,0.125646,0.020428,0.000278,1,l2,liblinear,"{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}",0.853379,0.873123,0.874786,0.876585,0.874086,0.870391,0.008581,10
3,4.037005,0.218976,0.021548,0.002862,1,l2,sag,"{'C': 1, 'penalty': 'l2', 'solver': 'sag'}",0.788725,0.786526,0.790583,0.789262,0.789121,0.788843,0.001316,25
4,4.537170,0.157007,0.019942,0.000307,1,l2,saga,"{'C': 1, 'penalty': 'l2', 'solver': 'saga'}",0.769933,0.767759,0.769408,0.770983,0.771630,0.769943,0.001340,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.008186,0.000402,0.000000,0.000000,500,elasticnet,newton-cg,"{'C': 500, 'penalty': 'elasticnet', 'solver': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
86,0.008170,0.000228,0.000000,0.000000,500,elasticnet,lbfgs,"{'C': 500, 'penalty': 'elasticnet', 'solver': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,55
87,0.006746,0.001336,0.000000,0.000000,500,elasticnet,liblinear,"{'C': 500, 'penalty': 'elasticnet', 'solver': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,56
88,0.006300,0.000094,0.000000,0.000000,500,elasticnet,sag,"{'C': 500, 'penalty': 'elasticnet', 'solver': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,84


In [22]:
import os

files = os.listdir("parameter_search/")
for file in files:
    df = pd.read_csv("parameter_search/"+file, index_col=0).sort_values(by=["rank_test_score"])
    print(file.split(".")[0], *df[df["rank_test_score"]==1]["params"], "\n")

knn_stdscaler_param_tuning {'knn__n_neighbors': 3} 

adaboost_param_tuning {'algorithm': 'SAMME.R'} 

decisiontree_param_tuning {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2, 'splitter': 'best'} 

knn_param_tuning {'n_neighbors': 3} 

rf_param_tuning {'max_depth': None, 'min_samples_split': 2} 

mlp_param_tuning {'early_stopping': True, 'hidden_layer_sizes': (200,), 'learning_rate': 'constant'} {'early_stopping': True, 'hidden_layer_sizes': (200,), 'learning_rate': 'adaptive'} 

svm_stdscaler_param_tuning {'svc__C': 500, 'svc__gamma': 'scale'} {'svc__C': 500, 'svc__gamma': 'auto'} 

sgd_param_tuning {'eta0': 0.5, 'learning_rate': 'adaptive', 'penalty': 'l2'} 

svm_param_tuning {'C': 50, 'gamma': 'auto'} {'C': 100, 'gamma': 'auto'} {'C': 200, 'gamma': 'auto'} {'C': 500, 'gamma': 'auto'} 

